# Notebook to test several inference methods

# Get data
TODO: get seperate test data


In [1]:
import numpy as np
import torch
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

seq_length = 512
sos_token = np.full((1, 88), 1)
pad_token = np.full((1, 88), 2)
pad_token = torch.tensor(pad_token, device=device)

In [2]:
from transformer_decoder_training.dataprep_transformer import dataprep_1
from data_preperation import dataset_snapshot
from sklearn.model_selection import train_test_split
# prepare data for dataset

#load data
dataset_as_snapshots = dataset_snapshot.process_dataset_multithreaded("/home/falaxdb/Repos/minus1/datasets/maestro_v3_split/hands_split_into_seperate_midis", 0.05, amount=20)
# filter snapshots to 88 piano notes
dataset_as_snapshots = dataset_snapshot.filter_piano_range(dataset_as_snapshots)

# Convert data into Chunks and add special tokens
data = dataprep_1.prepare_dataset(dataset_as_snapshots, seq_length, seq_length, sos_token)

print("Ammount of chunks:", len(data))
print("sequence of one chunk touple:", data[0][0].shape)

Processed dataset (40/40): 100%|██████████| 40/40 [00:01<00:00, 28.09it/s]


Processed 40 of 40 files
Ammount of chunks: 349
sequence of one chunk touple: (513, 88)


load model

In [3]:

from transformer_decoder_training.models.transformer_decoder_1 import Transformer



# Embedding Size
hidden_size = 256

# Number of transformer blocks
num_layers = 8

# MultiheadAttention Heads
num_heads = 8

# see: https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html#save-and-load-the-model
model = Transformer(num_emb=88, num_layers=num_layers, hidden_size=hidden_size, num_heads=num_heads).to(device)
model.load_state_dict(torch.load("/home/falaxdb/Repos/minus1/transformer_decoder_training/jupyter_notebooks/piano_tests/saved_models/model_1_notebook_v5.pth"))
model.eval()


Transformer(
  (embedding): Linear(in_features=88, out_features=256, bias=True)
  (pos_emb): SinusoidalPosEmb()
  (blocks): ModuleList(
    (0-7): 8 x TransformerBlock(
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (multihead_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=256, out_features=1024, bias=True)
        (1): ELU(alpha=1.0)
        (2): Linear(in_features=1024, out_features=256, bias=True)
      )
    )
  )
  (fc_out): Linear(in_features=256, out_features=88, bias=True)
  (sigmoid): Sigmoid()
)

## Obacht

Das Modell: `model_1_notebook_v5.pth` sagt nicht nur einen Token vorraus, sondern bei einem input von 513 Tokens werden auch 513 Vorhergesagt

evtl. weitere Infos: https://ai.stackexchange.com/questions/40140/how-is-the-next-token-predicted-in-transformers

In [4]:
from transformer_decoder_training.inference.inference_1 import inference

input_touple = data[300]
input_seq = input_touple[0]
input_seq = torch.tensor(input_seq).float().to(device)
# input must be of shape (batch_dim, seq_length, feature_dim
input_seq = torch.unsqueeze(input_seq, 0)

output_tokens = inference(model, input_seq, 512, 0.25, pad_token, device)

print(len(output_tokens))
#print(output_tokens)
      

512


# convert to Midi

In [5]:
# start token is existing
print(input_seq[0][0])

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:0')


In [6]:
# prepare data for midi conversion
generated_harmony = torch.cat(output_tokens, dim=0)
# convert to numpy
generated_harmony = generated_harmony.cpu().numpy()
original_melody = torch.squeeze(input_seq, dim=0).cpu().numpy()
original_harmony = input_touple[1]
# remove start token
original_melody = original_melody[1:]
original_harmony = original_harmony[1:]


print("original melody shape:", original_melody.shape)
print("original harmony shape:", original_harmony.shape)
print("generated harmony shape:", generated_harmony.shape)

original melody shape: (512, 88)
original harmony shape: (512, 88)
generated harmony shape: (512, 88)


In [7]:
from data_visualization import snapshot_to_midi

midi_tracks = [original_melody, original_harmony, generated_harmony]
track_names = ["o_melody", "o_harmony", "g_harmony"]

snapshot_to_midi.create_midi_from_snapshots(midi_tracks, track_names, 0.05, "/home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs", "model_1_notebook_v5.mid")

Processing track 0: o_melody with snapshot shape (512, 88)
Processing track 1: o_harmony with snapshot shape (512, 88)
Processing track 2: g_harmony with snapshot shape (512, 88)
MIDI file saved to /home/falaxdb/Repos/minus1/transformer_decoder_training/saved_files/midi_outputs
